## Challenge, Part 2

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [7]:
# Import file
city_data_df = pd.read_csv("./data/WeatherPy_Database.csv")

city_data_df

,city_id,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Rain (inches),Snow (inches),City Description,Country
0,0,Teshie,5.58,-0.11,78.80,78,100,9.17,0.0,0,overcast clouds,GH
1,1,Mount Gambier,-37.83,140.77,46.40,93,75,9.17,0.0,0,light intensity shower rain,AU
2,2,Clonakilty,51.62,-8.87,60.01,72,75,17.22,0.0,0,broken clouds,IE
3,3,San Rafael,-34.62,-68.33,50.25,48,99,7.78,0.0,0,overcast clouds,AR
4,4,Hasaki,35.73,140.83,73.40,94,75,9.17,0.0,0,broken clouds,JP
...,...,...,...,...,...,...,...,...,...,...,...,...
544,544,Olafsvik,64.89,-23.71,51.82,79,4,13.22,0.0,0,clear sky,IS
545,545,Tobermory,45.25,-81.67,82.99,57,12,1.01,0.0,0,few clouds,CA
546,546,Newport,51.59,-3.00,63.00,65,11,3.00,0.0,0,few clouds,GB
547,547,Novikovo,51.15,37.89,66.34,76,15,4.12,0.0,0,few clouds,RU


In [16]:
city_data_df.dtypes

city_id               int64
City                 object
Lat                 float64
Lon                 float64
Max Temp            float64
Humidity              int64
Cloudiness            int64
Wind Speed          float64
Rain (inches)       float64
Snow (inches)         int64
City Description     object
Country              object
dtype: object

In [8]:
# Configure Gmaps
gmaps.configure(api_key=g_key)

In [23]:
# Determine the min/max preferred temp
min_temp = float(input("What is the minimum temperature you prefer for your vacation?"))
max_temp = float(input("What is the maximum temperature you prefer for your vacation?"))

What is the minimum temperature you prefer for your vacation?50
What is the maximum temperature you prefer for your vacation?85


In [24]:
# Ask if rain is preferred
rain_amount = input("Do you want it to be raining? (yes/no)")

Do you want it to be raining? (yes/no)yes


In [25]:
# Ask if snow is preferred
snow_amount = input("Do you want it to be snowing? (yes/no)")

Do you want it to be snowing? (yes/no)no


In [26]:
if rain_amount == 'no' and snow_amount == 'no':
    filtered_cities_df = city_data_df.loc [(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] == 0) &
                                           (city_data_df["Snow (inches)"] == 0)]
elif rain_amount == 'no' and snow_amount == 'yes':
    filtered_cities_df = city_data_df.loc [(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] == 0) &
                                           (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_amount == 'yes' and snow_amount == 'no':
    filtered_cities_df = city_data_df.loc [(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] > 0.0) &
                                           (city_data_df["Snow (inches)"] == 0)]
else:
    filtered_cities_df = city_data_df.loc [(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] > 0.0) &
                                           (city_data_df["Snow (inches)"] > 0)]


In [27]:
filtered_cities_df.count()

city_id             51
City                51
Lat                 51
Lon                 51
Max Temp            51
Humidity            51
Cloudiness          51
Wind Speed          51
Rain (inches)       51
Snow (inches)       51
City Description    51
Country             51
dtype: int64

In [28]:
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Lat", "Lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lon,Hotel Name
6,Kostomuksha,RU,57.87,64.57,30.58,
19,Villablino,ES,75.99,42.94,-6.32,
25,Butaritari,KI,79.23,3.07,172.79,
39,Cidreira,BR,56.10,-30.18,-50.21,
49,Camopi,GF,79.43,3.17,-52.32,


In [43]:
# Set Params

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    
    params ["location"] = f"{lat}{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
H

In [44]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lon,Hotel Name
6,Kostomuksha,RU,57.87,64.57,30.58,
19,Villablino,ES,75.99,42.94,-6.32,
25,Butaritari,KI,79.23,3.07,172.79,
39,Cidreira,BR,56.10,-30.18,-50.21,
49,Camopi,GF,79.43,3.17,-52.32,


In [32]:
output_file = "./data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_file, index = "City_ID")

In [34]:
vacation_df= pd.read_csv("./data/WeatherPy_Vacation.csv")
vacation_df.head()

,Unnamed: 0,City,Country,Max Temp,Lat,Lon,Hotel Name
0,6,Kostomuksha,RU,57.87,64.57,30.58,NaN
1,19,Villablino,ES,75.99,42.94,-6.32,NaN
2,25,Butaritari,KI,79.23,3.07,172.79,NaN
3,39,Cidreira,BR,56.10,-30.18,-50.21,NaN
4,49,Camopi,GF,79.43,3.17,-52.32,NaN


In [36]:
# Add cities to a layer map.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [38]:
#  Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lon"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))